In [1]:
import pandas as pd
import zipfile
import seaborn as sns
import numpy as np
import pickle

In [2]:
data_path="../data/data_accuracy"

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd /content/drive/MyDrive/M5_forecasting

/content/drive/MyDrive/M5_forecasting


In [5]:
data_path="/content/drive/MyDrive/M5_forecasting/data/data_accuracy"

In [6]:
sales_train = pd.read_csv(data_path+"/sales_train_validation.csv")

In [7]:
list_levels = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

In [8]:
import itertools
itertools.combinations(list_levels,r=2)

In [9]:
all_combs = []
for r in range(1,4):
  all_combs.extend(list(itertools.combinations(list_levels,r=r)))

In [10]:
all_combs=all_combs[:-1]

In [11]:
all_combs.extend([('overall',)])

In [12]:
final_combs=[]
for combs in all_combs:
    combs2=[]
    for level in combs:
        combs2.append(level)
    final_combs.append(combs2)

In [13]:
overall = pd.DataFrame(sales_train[[a for a in sales_train.columns if "d_" in a]].sum())

In [14]:
overall.columns=['value']

In [15]:
overall.reset_index(inplace=True)

In [16]:
overall = overall.pivot_table(values=['value'],columns=['index'])

In [17]:
cols=[col for col in sales_train.columns if "d_" in col]

In [18]:
overall = overall[cols]

In [19]:
overall.reset_index(inplace=True)
overall.drop(['index'],axis=1,inplace=True)
overall.insert(0,"id","overall")
overall.insert(1,"idmap","overall")

In [20]:
def get_grouped_series(x,comb):
    if("overall" not in comb):
      temp_df=x.groupby(comb).sum().reset_index()
      temp_df.insert(0,"id",temp_df[comb].agg('|'.join, axis=1))
      temp_df.insert(1,"idmap","|".join(comb))
      temp_df.drop(comb,axis=1,inplace=True)
      return temp_df

In [21]:
grouped_time_series=[]

for comb in final_combs:
  grouped_time_series.append(get_grouped_series(sales_train,comb))


In [22]:
grouped_data = pd.concat(grouped_time_series,ignore_index=True)

In [23]:
grouped_data.shape

(161928, 1915)

In [24]:
grouped_data.drop_duplicates(cols,inplace=True)

In [25]:
grouped_data.shape

(42839, 1915)

In [27]:
grouped_data

,id,idmap,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,...,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,FOODS_1_001,item_id,6,6,4,6,7,18,10,4,11,10,6,5,12,15,16,18,11,10,12,7,17,9,9,8,5,14,26,8,12,13,5,11,10,14,11,8,22,14,...,3,6,3,10,12,6,2,6,3,9,18,4,5,2,3,5,5,2,3,6,4,10,1,5,4,6,8,0,6,3,4,4,30,7,5,3,6,2,16,6
1,FOODS_1_002,item_id,4,5,7,4,3,4,1,7,2,4,4,3,1,5,8,5,1,3,2,0,4,5,4,2,2,4,3,5,7,3,3,4,7,7,3,4,11,0,...,4,2,6,4,5,4,3,3,2,5,7,4,6,5,0,2,7,1,3,2,4,10,0,7,3,2,3,5,5,4,5,9,4,1,3,5,5,3,3,1
2,FOODS_1_003,item_id,14,8,3,6,3,8,13,10,11,6,11,10,3,17,20,14,10,9,3,14,7,23,16,7,12,6,5,7,10,6,10,6,8,3,9,18,18,11,...,8,6,4,8,6,5,8,6,7,5,7,6,6,10,8,5,6,2,7,3,2,4,4,5,8,7,7,4,4,1,7,3,5,6,3,4,4,3,11,5
3,FOODS_1_004,item_id,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,FOODS_1_005,item_id,34,32,13,20,10,21,18,20,25,41,16,23,30,21,45,61,27,20,13,13,17,26,30,34,14,36,25,9,37,39,18,13,18,19,17,24,40,16,...,17,17,18,10,22,19,17,12,31,37,45,27,13,5,9,28,42,17,15,28,11,16,14,22,28,19,24,30,25,24,16,14,14,18,18,27,12,15,38,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48939,HOBBIES|TX,cat_id|state_id,879,870,526,809,501,831,390,785,794,524,556,618,648,598,747,837,555,519,589,612,518,862,812,535,635,531,645,586,904,543,349,459,450,453,340,534,927,455,...,949,989,899,1325,1167,826,887,1107,913,1036,1184,1107,920,829,950,878,1185,1423,1284,999,875,998,998,1080,1145,1302,1125,1045,1016,1055,1107,1124,1461,830,1025,980,882,1056,1222,1266
48940,HOBBIES|WI,cat_id|state_id,1083,926,684,455,132,930,1240,1215,623,583,610,604,669,836,1048,712,622,664,640,627,712,1263,589,452,431,826,614,733,1040,476,358,418,416,578,461,496,901,836,...,834,688,920,1347,1027,803,695,651,790,1345,1254,716,881,796,717,929,1308,1264,1168,959,816,722,865,1149,1442,1072,800,860,961,865,1021,1334,1029,787,913,846,881,1014,1198,1060
48941,HOUSEHOLD|CA,cat_id|state_id,2292,2382,1692,1778,1566,1607,1932,2754,2237,1677,1591,1490,1674,1974,2577,2653,1505,1789,1637,1475,1783,2640,2801,2487,1772,1689,1589,1814,2465,2468,1779,1737,1728,1776,1828,2539,2869,1829,...,3515,3367,4009,4743,5120,4114,3548,3482,3553,4226,5411,5280,3907,3653,3521,4046,4734,5920,6505,4518,3809,3556,3725,4306,5967,6137,4306,3627,3611,3697,3865,5294,5847,4129,3847,3639,3419,4089,5622,6024
48942,HOUSEHOLD|TX,cat_id|state_id,1706,1730,1128,1102,809,1108,1165,1600,1662,1290,1040,970,988,1234,1670,2006,1039,1230,1246,1277,1512,2113,2243,1499,1322,1264,1257,1248,1852,1904,1383,1293,1226,1202,1275,1801,1821,1241,...,2768,2702,2836,3496,3779,2674,2538,2796,2387,2889,3323,2904,2798,2282,2444,2697,3154,3675,3641,2815,2743,2551,2678,2762,3435,3502,2811,2419,2232,2622,2699,3392,3531,2204,2831,2378,2553,2720,3195,3251


In [29]:
grouped_data.append(overall,ignore_index=True)

index,id,idmap,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,...,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,FOODS_1_001,item_id,6,6,4,6,7,18,10,4,11,10,6,5,12,15,16,18,11,10,12,7,17,9,9,8,5,14,26,8,12,13,5,11,10,14,11,8,22,14,...,3,6,3,10,12,6,2,6,3,9,18,4,5,2,3,5,5,2,3,6,4,10,1,5,4,6,8,0,6,3,4,4,30,7,5,3,6,2,16,6
1,FOODS_1_002,item_id,4,5,7,4,3,4,1,7,2,4,4,3,1,5,8,5,1,3,2,0,4,5,4,2,2,4,3,5,7,3,3,4,7,7,3,4,11,0,...,4,2,6,4,5,4,3,3,2,5,7,4,6,5,0,2,7,1,3,2,4,10,0,7,3,2,3,5,5,4,5,9,4,1,3,5,5,3,3,1
2,FOODS_1_003,item_id,14,8,3,6,3,8,13,10,11,6,11,10,3,17,20,14,10,9,3,14,7,23,16,7,12,6,5,7,10,6,10,6,8,3,9,18,18,11,...,8,6,4,8,6,5,8,6,7,5,7,6,6,10,8,5,6,2,7,3,2,4,4,5,8,7,7,4,4,1,7,3,5,6,3,4,4,3,11,5
3,FOODS_1_004,item_id,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,FOODS_1_005,item_id,34,32,13,20,10,21,18,20,25,41,16,23,30,21,45,61,27,20,13,13,17,26,30,34,14,36,25,9,37,39,18,13,18,19,17,24,40,16,...,17,17,18,10,22,19,17,12,31,37,45,27,13,5,9,28,42,17,15,28,11,16,14,22,28,19,24,30,25,24,16,14,14,18,18,27,12,15,38,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42835,HOBBIES|WI,cat_id|state_id,1083,926,684,455,132,930,1240,1215,623,583,610,604,669,836,1048,712,622,664,640,627,712,1263,589,452,431,826,614,733,1040,476,358,418,416,578,461,496,901,836,...,834,688,920,1347,1027,803,695,651,790,1345,1254,716,881,796,717,929,1308,1264,1168,959,816,722,865,1149,1442,1072,800,860,961,865,1021,1334,1029,787,913,846,881,1014,1198,1060
42836,HOUSEHOLD|CA,cat_id|state_id,2292,2382,1692,1778,1566,1607,1932,2754,2237,1677,1591,1490,1674,1974,2577,2653,1505,1789,1637,1475,1783,2640,2801,2487,1772,1689,1589,1814,2465,2468,1779,1737,1728,1776,1828,2539,2869,1829,...,3515,3367,4009,4743,5120,4114,3548,3482,3553,4226,5411,5280,3907,3653,3521,4046,4734,5920,6505,4518,3809,3556,3725,4306,5967,6137,4306,3627,3611,3697,3865,5294,5847,4129,3847,3639,3419,4089,5622,6024
42837,HOUSEHOLD|TX,cat_id|state_id,1706,1730,1128,1102,809,1108,1165,1600,1662,1290,1040,970,988,1234,1670,2006,1039,1230,1246,1277,1512,2113,2243,1499,1322,1264,1257,1248,1852,1904,1383,1293,1226,1202,1275,1801,1821,1241,...,2768,2702,2836,3496,3779,2674,2538,2796,2387,2889,3323,2904,2798,2282,2444,2697,3154,3675,3641,2815,2743,2551,2678,2762,3435,3502,2811,2419,2232,2622,2699,3392,3531,2204,2831,2378,2553,2720,3195,3251
42838,HOUSEHOLD|WI,cat_id|state_id,1691,1522,1107,985,354,1183,1479,1841,1076,1089,930,1086,1054,1377,1700,1648,1012,1072,1039,1227,1548,1856,1233,872,905,1114,1139,1312,1598,1557,1241,1441,1199,1400,1304,1694,1482,1217,...,2097,2220,2558,3208,3162,2320,2007,1824,1837,3360,3518,2297,2496,2285,2044,2390,4231,3930,3743,2399,2411,2410,2615,2637,3865,3246,2503,2215,1905,1968,2757,3035,2945,2252,2157,2222,2391,2919,3431,3183


In [31]:
grouped_data.to_csv(data_path+"/grouped_data.csv")

In [ ]:
##end of file